### This is a notebook, which generates predictions for SpaceTitanic passenger survival competition.
It's accuracy is up to 0.8045, a top 15\% result.

### Outline:
0. Load libraries and custom functions.
1. Load data.
2. Preliminary data analysis: explore features and a target, delete unneeded features, create new features.
3. Train-test split.
4. Missing values. In some cases it may be useful to explore skew and perform log-transform before imputing missing values.
5. Feature engineering. Transform skewed variables, do OHC and scaling.
6. Fit models.
7. Evaluate models.
8. Feature importance, error analysis. Based on the results, go to 2. and iterate.
9. Make predictions.

Other notes:
- Adding more features seems to hurt results.
- I tried adding features based on group, room, dummies for missinf values - none helped.

In [ ]:
# 0. Load libraries #

import numpy as np
import pandas as pd
import os, time, warnings
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')

# Load custom pre-processing functions:

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()


def fillna_mp_i1(df_train, df_test, df_pred, num_features, cat_features, num_fill='median', cat_fill='mode'):
    """This function speeds up filling missing values for 3 main datasets using different imputation methods.
    Later may replace it with some subclass.
    Example: fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)"""
    # set df_pred to None if it does not exist
    if not ((cat_fill=='mode') and (num_fill=='median')):
        print ('Imputation method not Implemented yet!')
        return None
    
    df_train[num_features] = df_train[num_features].fillna(value=df_train[num_features].median())
    df_test[num_features] = df_test[num_features].fillna(value=df_train[num_features].median())
    df_train[cat_features] = df_train[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_test[cat_features] = df_test[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    if (df_pred is not None):
        df_pred[num_features] = df_pred[num_features].fillna(value=df_train[num_features].median())
        df_pred[cat_features] = df_pred[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_train[num_features+cat_features].count
    
    all_good = (
    (np.prod(df_train[num_features+cat_features].shape)==df_train[num_features+cat_features].count().sum()) and 
    (np.prod(df_test[num_features+cat_features].shape) == df_test[num_features+cat_features].count().sum()) and 
    (np.prod(df_pred[num_features+cat_features].shape) == df_pred[num_features+cat_features].count().sum()))
    if (all_good):
        print('Missing values imputed successfully')
    else:
        print('There are still some missing values...')
    
def add_misDummy_mp_i1(df_train, df_test, df_pred, features):
    """This function creates new dummy columns for missing features.
    Example: add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age'])"""
    # set df_pred to None if it does not exist
    for feature_name in features:
        misColName = 'mis'+feature_name
        df_train.loc[df_train[feature_name].isnull(), misColName]=1
        df_train.loc[df_train[feature_name].notnull(), misColName]=0
        df_test.loc[df_test[feature_name].isnull(), misColName]=1
        df_test.loc[df_test[feature_name].notnull(), misColName]=0
        if (df_pred is not None):
            df_pred.loc[df_pred[feature_name].isnull(), misColName]=1
            df_pred.loc[df_pred[feature_name].notnull(), misColName]=0
   

def discretize_mp_i1(df_train, df_test, df_pred, feature, ntiles, delete_feature=False):
    """This function divides a continuous feature into quantile groups.
    Example: discretize_mp_i1(X_train, X_test, X_pred, 'Age', 15)"""
    # set df_pred to None if it does not exist
    _,bin = pd.qcut(df_train[feature], ntiles, retbins = True, labels = False, duplicates = 'drop')
    df_train[feature+'Ntile'] = pd.cut(df_train[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    df_test[feature+'Ntile'] = pd.cut(df_test[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (df_pred is not None):
        df_pred[feature+'Ntile'] = pd.cut(df_pred[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (delete_feature==True):
        df_train.drop(columns=[feature], inplace=True)
        df_test.drop(columns=[feature], inplace=True)
        df_pred.drop(columns=[feature], inplace=True)
    print('Discretized ',feature, ' into ', len(bin)-1, ' bins')


def log_transformer_mp_i1(df_train, df_test, df_pred, feature_subset=False, min_skew=3):
    """This function divides a continuous feature into quantile groups.
    Example: log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)"""
    # set df_pred to None if it does not exist
    if (feature_subset==False):
        features_totransform = df_train.columns
    else:
        features_totransform = feature_subset.copy()
    skewed_vars = list(df_train.skew()[abs(df_train.skew())>min_skew].index)
    for col in list(set(skewed_vars)&set(features_totransform)):
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
        if (df_pred is not None):
            df_pred[col] = np.log1p(df_pred[col])
    print('Skewed columns log-transformed: ', list(set(skewed_vars)&set(features_totransform)))
    
    
def add_dummyfeatures(df_train, df_test, df_pred, feature_dict):
    """This function adds dummy feature when some feature is equal to value, specified in a dictionary.
    Example: add_dummyfeatures(X_train, X_test, X_pred, {'RoomService':0, 'Spa':0, 'VRDeck':0, 'ShoppingMall':0})"""
    input_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    for i in range(len(list(feature_dict.items()))):
        feature,value = list(feature_dict.keys())[i], list(feature_dict.values())[i]
        df_train.loc[df_train[feature]==value,(str(feature)+str(value))]=1
        df_train.loc[df_train[feature]!=value,(str(feature)+str(value))]=0
        df_test.loc[df_test[feature]==value,(str(feature)+str(value))]=1
        df_test.loc[df_test[feature]!=value,(str(feature)+str(value))]=0
        df_pred.loc[df_pred[feature]==value,(str(feature)+str(value))]=1
        df_pred.loc[df_pred[feature]!=value,(str(feature)+str(value))]=0
    output_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    print(output_dimensions-input_dimensions, ' variables created') 
    
    
# 1. Load data #

time0 = time.time()

path = '../input/spaceship-titanic/train.csv'
df = pd.read_csv(path) 
df0 = df.copy()

#df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'],inplace=True)
pred=pd.read_csv('../input/spaceship-titanic/test.csv')
pred0 = pred.copy()
#pred.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'],inplace=True)

print(df.shape, pred.shape)
df.head()

# 2. pEDA #
df[['Deck', 'Room', 'Side']] = df['Cabin'].str.split('/', 2, expand=True)
df[['Group_Id', 'Passeng_Id']] = df['PassengerId'].str.split('_', 1, expand=True)
pred[['Deck', 'Room', 'Side']] = pred['Cabin'].str.split('/', 2, expand=True)
pred[['Group_Id', 'Passeng_Id']] = pred['PassengerId'].str.split('_', 1, expand=True)
df.drop(columns = ['PassengerId', 'Cabin', 'Name', 'Passeng_Id'], inplace = True)
pred.drop(columns = ['PassengerId', 'Cabin', 'Name', 'Passeng_Id'], inplace = True)
df

sns.barplot(x='Transported', y='CryoSleep', data=df)
plt.show()
sns.histplot(x='RoomService', data=df, bins=50)
plt.show()
sns.barplot(x='Transported', y='RoomService', data=df)
plt.show()
sns.histplot(x='Spa', data=df, bins=50)
plt.show()
sns.barplot(x='Transported', y='Spa', data=df)
plt.show()
sns.histplot(x='VRDeck', data=df, bins=50)
plt.show()
sns.barplot(x='Transported', y='VRDeck', data=df)
plt.show()
sns.barplot(x='Transported', y='CryoSleep', data=df)
plt.show()

# 3. Train-test split #

train_y = df[['Transported']]
train_x = df.drop(columns = ['Transported'])
X_pred = pred.copy()

cat_cols = [col for col in train_x.columns if train_x[col].nunique() in range(2,10)]
num_cols = list(set(train_x.columns)-set(cat_cols)-{'Group_Id', 'Room'})

print('categorical features: ', cat_cols, 'numerical features: ', num_cols)

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.01, random_state=24)
print(X_train.shape, X_test.shape, y_train.shape, X_pred.shape)

X_train.info()

# add room and group variables:

roomsize = X_train.groupby('Room').size().to_frame()
roomsize.reset_index(inplace=True)
roomsize.columns = ['Room', 'Num_ppl_room']
roomsize.sort_values('Num_ppl_room', inplace = True)
X_train = pd.merge(X_train, roomsize, on='Room', how='left')
X_test = pd.merge(X_test, roomsize, on='Room', how='left')
X_pred = pd.merge(X_pred, roomsize, on='Room', how='left')
groupsize = X_train.groupby('Group_Id').size().to_frame()
groupsize.reset_index(inplace=True)
groupsize.columns = ['Group_Id', 'Num_ppl_group']
groupsize.sort_values('Num_ppl_group', inplace = True)
X_train = pd.merge(X_train, groupsize, on='Group_Id', how='left')
X_test = pd.merge(X_test, groupsize, on='Group_Id', how='left')
X_pred = pd.merge(X_pred, groupsize, on='Group_Id', how='left')

num_cols.extend(['Num_ppl_room', 'Num_ppl_group'])


# 4. Missing values #

#add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'RoomService',
#'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Deck', 'Side'])
fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)
X_train.info()

# 5. Feature engineering #

# add useful variables:
add_dummyfeatures(X_train, X_test, X_pred, {'RoomService':0, 'Spa':0, 'VRDeck':0, 'ShoppingMall':0})
# ZeroFC dummy seem to hurt results...

# add more features 

X_train.drop(columns=['Room', 'Group_Id'], inplace= True)
X_train.loc[(X_train['RoomService']==0)&(X_train['FoodCourt']==0)&(X_train['ShoppingMall']==0)&(X_train['Spa']==0)&(X_train['VRDeck']==0),'ZeroService'] = 1
X_train.loc[X_train.ZeroService.isnull(), 'ZeroService'] = 0

X_test.drop(columns=['Room', 'Group_Id'], inplace= True)
X_test.loc[(X_test['RoomService']==0)&(X_test['FoodCourt']==0)&(X_test['ShoppingMall']==0)&(X_test['Spa']==0)&(X_test['VRDeck']==0),'ZeroService'] = 1
X_test.loc[X_test.ZeroService.isnull(), 'ZeroService'] = 0

X_pred.drop(columns=['Room', 'Group_Id'], inplace= True)
X_pred.loc[(X_pred['RoomService']==0)&(X_pred['FoodCourt']==0)&(X_pred['ShoppingMall']==0)&(X_pred['Spa']==0)&(X_pred['VRDeck']==0),'ZeroService'] = 1
X_pred.loc[X_pred.ZeroService.isnull(), 'ZeroService'] = 0


discretize_mp_i1(X_train, X_test, X_pred, 'Age', 10)
cat_cols.extend(['AgeNtile', 'RoomService0', 'Spa0', 'VRDeck0', 'ShoppingMall0', 'ZeroService'])

X_train.describe()
X_train[:5]

log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)

feature_transformer = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(sparse = False, handle_unknown="ignore"), cat_cols),
    ])

X_train = pd.DataFrame(feature_transformer.fit_transform(X_train), columns=feature_transformer.get_feature_names_out())
X_test = pd.DataFrame(feature_transformer.transform(X_test), columns=feature_transformer.get_feature_names_out())
X_pred = pd.DataFrame(feature_transformer.transform(X_pred), columns=feature_transformer.get_feature_names_out())

fewfeatures = ['cat__HomePlanet_Europa', 'cat__CryoSleep_True', 'cat__Deck_B', 'cat__Deck_C', 'cat__Side_P', 'cat__Destination_TRAPPIST-1e', 
               'num__RoomService', 'num__FoodCourt', 'num__ShoppingMall', 'num__Spa', 'num__VRDeck', 'num__Age']

print(X_train.shape)

In [ ]:
# 6. Fit models #

lr = LogisticRegression()
param_grid = {'C':[0.3, 0.6, 1, 2]}
lrm = GridSearchCV(lr, param_grid, cv=8)
lrm.fit(X_train, y_train)
print('Logistic ', lrm.best_params_, accuracy_score(y_train, lrm.predict(X_train)))

time1 = time.time()
svm = SVC()
param_grid = {'C':[1, 2, 3]}
svmm = GridSearchCV(svm, param_grid, cv=2)
svmm.fit(X_train, y_train)
print('SVM ', svmm.best_params_, accuracy_score(y_train, svmm.predict(X_train)), time.time()-time1)

knn = KNeighborsClassifier(n_jobs=-1)
param_grid = dict(n_neighbors=range(5, 36, 5))
knnm = GridSearchCV(knn, param_grid, cv=2)
knnm.fit(X_train[fewfeatures], y_train)
print('KNN ', knnm.best_params_, accuracy_score(y_train, knnm.predict(X_train[fewfeatures])))

time1 = time.time()
rf = RandomForestClassifier(n_jobs=-1)
param_grid = {'n_estimators':[100], 'max_depth':[6,7,8], 'max_features':[6,7,8]}
rfm = GridSearchCV(rf, param_grid, cv=2)
rfm.fit(X_train, y_train)
print('RF ', rfm.best_params_, rfm.best_score_, accuracy_score(y_train, rfm.predict(X_train)), time.time()-time1)

time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=3, n_jobs=-1)
# use 'gpu_hist' for more than 10,000 examples.
param_grid = {'n_estimators':[100,200], 'max_depth':[2,3,4,5], 'eta':[0.04, 0.06, 0.08], 
              'subsample':[0.6],'colsample_bytree':[0.6]}
#'subsample':[0.4, 0.6],'colsample_bytree':[0.6, 0.8]
xgbm = GridSearchCV(xgb, param_grid, cv=2)
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, accuracy_score(y_train, xgbm.predict(X_train)), time.time()-time1)

# 7. accuracy #

print('Out of Sample:')
print('Logistic ', accuracy_score(y_test, lrm.predict(X_test)))
print('SVM ', accuracy_score(y_test, svmm.predict(X_test)))
print('KNN ', accuracy_score(y_test, knnm.predict(X_test[fewfeatures])))
print('RF ', accuracy_score(y_test, rfm.predict(X_test)))
print('XGB ', accuracy_score(y_test, xgbm.predict(X_test)))
print('Total time ', time.time()-time0)

# VotingClassifier:

estimator = []
#estimator.append(('LR', LogisticRegression(C=0.5)))
estimator.append(('SVM', SVC(C=2, probability = True)))
#estimator.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
estimator.append(('RF', RandomForestClassifier(max_depth=8, max_features=7, n_estimators=100)))
estimator.append(('XGB', XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=3, n_jobs=-1,
                                       eta=0.045, max_depth=4, n_estimators=200, 
                                       subsample=0.6, colsample_bytree=0.6)))
vot_soft = VotingClassifier(estimators = estimator, voting ='soft')
vot_soft.fit(X_train, y_train)
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(X_train, y_train)
print('VotingClassifier3 in sample', accuracy_score(y_train, vot_soft.predict(X_train)), accuracy_score(y_train, vot_hard.predict(X_train)))
print('VotingClassifier3 out of sample', accuracy_score(y_test, vot_soft.predict(X_test)), accuracy_score(y_test, vot_hard.predict(X_test)))

In [ ]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=5)
# use 'gpu_hist' for more than 10,000 examples.
param_grid = {'n_estimators':[200], 'max_depth':[4,5], 'eta':[0.04, 0.06], 
              'subsample':[0.6],'colsample_bytree':[0.6]}
#'subsample':[0.4, 0.6],'colsample_bytree':[0.6, 0.8]
xgbm = GridSearchCV(xgb, param_grid, cv=2)
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, accuracy_score(y_train, xgbm.predict(X_train)), time.time()-time1)

In [ ]:
# 8. feature importance #

results = permutation_importance(xgbm, X_test, y_test, scoring='accuracy', n_jobs=-1)
fi_lr = pd.DataFrame({'col':X_test.columns, 'FI':results.importances_mean})
fi_lr.sort_values('FI', ascending = False)

In [ ]:
# 9. predictions #

submission_df_vc = pd.DataFrame({'PassengerId': pred0.PassengerId, 'Transported': vot_hard.predict(X_pred)}, columns=['PassengerId', 'Transported'])
submission_df_svm = pd.DataFrame({'PassengerId': pred0.PassengerId, 'Transported': svmm.predict(X_pred)}, columns=['PassengerId', 'Transported'])
submission_df_rf = pd.DataFrame({'PassengerId': pred0.PassengerId, 'Transported': rfm.predict(X_pred)}, columns=['PassengerId', 'Transported'])
submission_df_bt = pd.DataFrame({'PassengerId': pred0.PassengerId, 'Transported': xgbm.predict(X_pred)}, columns=['PassengerId', 'Transported'])

submission_df_bt.Transported = np.array([bool(x) for x in submission_df_bt.Transported])

submission_df_vc.to_csv('KP11_vc.csv',index=False)
submission_df_svm.to_csv('KP11_svm.csv',index=False)
submission_df_rf.to_csv('KP11_rf.csv',index=False)
submission_df_bt.to_csv('KP11_bt.csv',index=False)

os.chdir(r'/kaggle/working')

from IPython.display import FileLink
FileLink(r'KP11_bt.csv')